<a href="https://colab.research.google.com/github/sagihaider/DANN_2020/blob/master/main_DANN_EEG_MEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sagihaider/DANN_2020.git

Cloning into 'DANN_2020'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 155 (delta 14), reused 135 (delta 5), pack-reused 0
Receiving objects: 100% (155/155), 9.36 MiB | 17.56 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [0]:
import os
import time
# !pip install keras --upgrade
import numpy as np
# from keras import backend as K
#from tensorflow.keras import backend as K
import keras.backend.tensorflow_backend as K
from scipy.stats import sem
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tqdm import tqdm

from DANN_2020.function.utils import entropy1
import scipy.io as spio
from numpy import zeros
import pandas as pd
import numpy as np
from argparse import ArgumentParser
import matplotlib.pyplot as plt
from DANN_2020.function.sklearn_nn import NNClassifier
#from sklearn_nn_evo import NNClassifier
from DANN_2020.function.bagging import BaggingClassifierDomainAdaptation as BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
#from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.svm import NuSVC
from sklearn.ensemble import RandomForestClassifier

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

N_EPOCHS = 2000
N_THREADS = 1

# K.set_session(K.Session(config = K.ConfigProto(intra_op_parallelism_threads=N_THREADS, inter_op_parallelism_threads=N_THREADS)))

# K.set_session(K.Session(
#     config = K
#     config=K.tf.ConfigProto(intra_op_parallelism_threads=N_THREADS, inter_op_parallelism_threads=N_THREADS)))


# parser = ArgumentParser()
# parser.add_argument("-s", "--subject", dest="subject",
#                     help="subject number", metavar="int")

# args = parser.parse_args()

In [0]:
cols = 2
rows = 9
results_dann = zeros([rows, cols])

for x in (range(1, 10)):
  subject = x
  fName = 'DANN_2020/Data/EEG/A0' + str(subject) + '_FBCSP.mat'  # Load Data
  print(fName)
  mat = spio.loadmat(fName)
  Xs = mat['Train_X']
  Xt = mat['Test_X']
  a = int(len(Xs) / 2)
  b = int(len(Xt) / 2)
  ys = np.concatenate([np.zeros(a, dtype=int), np.ones(a, dtype=int)])
  yt = np.concatenate([np.zeros(b, dtype=int), np.ones(b, dtype=int)])
  clf = BaggingClassifier(NNClassifier(N_EPOCHS, Xt, enable_dann=True, batch_size=64), n_estimators=10, bootstrap_features=False, bootstrap=False, verbose=1000)
  #clf = AdaBoostClassifier(NNClassifier(N_EPOCHS, Xt, enable_dann=True),  n_estimators=10,  learning_rate=0.2)
  clf.fit(Xs, ys)

  ys_hat = clf.predict(Xs)
  yt_hat = clf.predict(Xt)
  s_acc = accuracy_score(ys, ys_hat)
  t_acc = accuracy_score(yt, yt_hat)
  t2_acc = accuracy_score(yt, clf.predict(Xt + np.random.normal(0,0.3, size = Xt.shape)))

  # print("\033[1;32;40m ")
  print("Subject Accuracy", subject, s_acc, t_acc, t2_acc)
  # print("\033[0;37;40m")
  
  results_dann[x - 1, 0] = s_acc
  results_dann[x - 1, 1] = t_acc


print(results_dann)

DANN_2020/Data/EEG/A01_FBCSP.mat
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Building estimator 1 of 10 for this parallel run (total 10)...


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Building estimator 2 of 10 for this parallel run (total 10)...


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Building estimator 3 of 10 for this parallel run (total 10)...


...

Building estimator 9 of 10 for this parallel run (total 10)...


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Building estimator 10 of 10 for this parallel run (total 10)...


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Subject Accuracy 9 0.8571428571428571 0.9097222222222222 0.9166666666666666
[[0.9        0.875     ]
 [0.78571429 0.53472222]
 [0.97142857 0.97222222]
 [0.87

In [0]:
df_dann=pd.DataFrame(results_dann)
df_dann.to_csv('dann_results.csv')

## MEG-based Single Trial Classification

In [9]:
cols = 2
rows = 18
results_dann_MEG = zeros([rows, cols])

sub_count = 0

sub_list = [1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
# sub_list = [1]

for x in sub_list:
  subject = x
  sub_count = sub_count + 1
  fName = 'DANN_2020/Data/MEG/P00' + str(subject) + 'ClassComb_1_2_Features.mat'  # Load Data
  print(fName)
  mat = spio.loadmat(fName)
  print(mat.keys())
  Xs=np.empty((100, 0))
  Xt=np.empty((100, 0))
  tr=mat['features_tr'] # fetch training data
  ts=mat['features_ts'] # fetch test data
  print(tr.shape)
  print(ts.shape)
  nrow, ncell = tr.shape
  for i in (range(0, ncell)):
    iband=tr[0,i]
    ncomp=iband[:,[0,5]]#
    Xs = np.append(Xs, ncomp, axis=1)

  nrow, ncell = ts.shape
  for i in (range(0, ncell)):
    iband=ts[0,i]
    ncomp=iband[:,[0,5]]#
    Xt = np.append(Xt, ncomp, axis=1)

  ys = mat['Y_tr']
  yt = mat['Y_ts']

  clf = BaggingClassifier(NNClassifier(N_EPOCHS, Xt, enable_dann=True, batch_size=64), n_estimators=10, bootstrap_features=False, bootstrap=False, verbose=1000)
  #clf = AdaBoostClassifier(NNClassifier(N_EPOCHS, Xt, enable_dann=True),  n_estimators=10,  learning_rate=0.2)
  clf.fit(Xs, ys)

  ys_hat = clf.predict(Xs)
  yt_hat = clf.predict(Xt)
  s_acc = accuracy_score(ys, ys_hat)
  t_acc = accuracy_score(yt, yt_hat)
  t2_acc = accuracy_score(yt, clf.predict(Xt + np.random.normal(0,0.3, size = Xt.shape)))

  # print("\033[1;32;40m ")
  print("Subject Accuracy", subject, s_acc, t_acc, t2_acc)
  # print("\033[0;37;40m")
  
  results_dann_MEG[sub_count - 1, 0] = s_acc
  results_dann_MEG[sub_count - 1, 1] = t_acc


print(results_dann_MEG)
results_dann_MEG=pd.DataFrame(results_dann_MEG)
print(results_dann_MEG)
results_dann_MEG.to_csv('dann_results_MEG.csv')
  

DANN_2020/Data/MEG/P001ClassComb_1_2_Features.mat
dict_keys(['__header__', '__version__', '__globals__', 'Y_tr', 'Y_ts', 'features_tr', 'features_ts', 'highpass_freq', 'lowpass_freq', 'toi'])
(1, 10)
(1, 10)


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Building estimator 1 of 10 for this parallel run (total 10)...


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Building estimator 2 of 10 for this parallel run (total 10)...


...

Building estimator 9 of 10 for this parallel run (total 10)...


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Building estimator 10 of 10 for this parallel run (total 10)...


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Subject Accuracy 20 1.0 0.64 0.62
[[1.   0.54]
 [1.   0.49]
 [1.   0.78]
 [1.   0.54]
 [0.55 0.5 ]
 [1.   0.5 ]
 [0.99 0.64]
 [0.81 0.5 ]
 [0.65 0.5 ]
 [1.  